# YOLOv3를 이용한 실시간 객체 탐지

## 데이터 생성기 만들기고 옵니마이저 지정하기

In [1]:
import tensorflow as tf
from config import *
from data import DataGenerator 

trainset = DataGenerator(data_path=TRAIN_DATA_PATH,
                         annot_path=TRAIN_ANNOT_PATH,
                         class_label_path=CLASS_LABEL_PATH)
testset = DataGenerator(data_path=TEST_DATA_PATH, 
                        annot_path=TEST_ANNOT_PATH,
                        class_label_path=CLASS_LABEL_PATH)
steps_per_epoch = len(trainset)
global_steps = tf.Variable(1, trainable=False, dtype=tf.int64) 
warmup_steps = WARMUP_EPOCHS * steps_per_epoch
total_steps = EPOCHS * steps_per_epoch

optimizer = tf.keras.optimizers.Adam()

## 학습시키기

In [2]:
import os
from config import *
from bbox_iou import bbox_iou, bbox_giou
from yolov3 import Create_YOLOv3
from train import *


yolo = Create_YOLOv3(num_class=NUM_CLASS, train_mode=True)

best_val_loss = 99999
save_directory = os.path.join(CHECKPOINTS_FOLDER, MODEL_NAME)

for epoch in range(EPOCHS):
    for image_data, target in trainset:
        results = train_step(yolo, image_data, target)
        cur_step = results[0] % steps_per_epoch
        print("epoch:{:2.0f} step:{:5.0f}/{}, lr:{:.6f}, giou_loss:{:7.2f}, conf_loss:{:7.2f}, prob_loss:{:7.2f}, total_loss:{:7.2f}".format(epoch, cur_step, steps_per_epoch, results[1], results[2], results[3], results[4], results[5]))
 
    if len(testset) == 0: 
        print("configure TEST options to validate model")
        yolo.save_weights(save_directory)
        continue 

    count = 0
    giou_val, conf_val, prob_val, total_val = 0, 0, 0, 0 

    for image_data, target in testset:
        results = validate_step(yolo, image_data, target)
        count += 1
        giou_val += results[0]
        conf_val += results[1]
        prob_val += results[2]
        total_val += results[3]

    # validation loss 저장 
    with validate_writer.as_default():
        tf.summary.scalar("validate_loss/total_val", total_val / count, step=epoch)
        tf.summary.scalar("validate_loss/giou_val", giou_val / count, step=epoch)
        tf.summary.scalar("validate_loss/conf_val", conf_val / count, step=epoch)
        tf.summary.scalar("validate_loss/prob_val", prob_val / count, step=epoch)
    validate_writer.flush()
    # print("giou_val_loss:{:7.2f}, conf_val_loss:{:7.2f}, prob_val_loss:{:7.2f}, total_val_loss:{:7.2f}\n\n".format( giou_val / count, conf_val / count, prob_val / count, total_val / count))

    if SAVE_CHECKPOINT and not SAVE_BEST_ONLY:
        save_directory = os.path.join(CHECKPOINTS_FOLDER,  MODEL_NAME + "_epoch_{:03d}_val_loss_{:7.2f}".format(epoch, total_val / count))

    # print(best_val_loss, total_val / count)

    if SAVE_BEST_ONLY:
        if best_val_loss > total_val / count:
            best_val_loss = total_val / count
            yolo.save_weights(save_directory)

GPUs [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
epoch: 0 step:    2/192, lr:0.000001, giou_loss:  54.33, conf_loss:1755.11, prob_loss:  98.20, total_loss:1907.64
epoch: 0 step:    3/192, lr:0.000001, giou_loss:  63.52, conf_loss:1595.58, prob_loss: 111.94, total_loss:1771.04
epoch: 0 step:    4/192, lr:0.000001, giou_loss:  37.68, conf_loss:1582.90, prob_loss:  66.78, total_loss:1687.37
epoch: 0 step:    5/192, lr:0.000001, giou_loss:  46.01, conf_loss:1577.74, prob_loss:  82.24, total_loss:1705.99
epoch: 0 step:    6/192, lr:0.000002, giou_loss:  43.97, conf_loss:1581.36, prob_loss:  77.39, total_loss:1702.72
epoch: 0 step:    7/192, lr:0.000002, giou_loss:  56.73, conf_loss:1589.67, prob_loss:  95.85, total_loss:1742.24
epoch: 0 step:    8/192, lr:0.000002, giou_loss:  47.06, conf_loss:1586.19, prob_loss:  79.18, total_loss:1712.43
epoch: 0 step:    9/192, lr:0.000002, giou_loss:  44.62, conf_loss:1570.14, prob_loss:  77.89, total_loss:1692.66
epoch: 0 step:  

epoch: 0 step:   74/192, lr:0.000019, giou_loss:  45.69, conf_loss: 982.38, prob_loss:  85.76, total_loss:1113.83
epoch: 0 step:   75/192, lr:0.000020, giou_loss:  20.01, conf_loss: 941.37, prob_loss:  38.95, total_loss:1000.33
epoch: 0 step:   76/192, lr:0.000020, giou_loss:  32.68, conf_loss: 935.47, prob_loss:  64.64, total_loss:1032.79
epoch: 0 step:   77/192, lr:0.000020, giou_loss:  25.71, conf_loss: 921.42, prob_loss:  52.18, total_loss: 999.32
epoch: 0 step:   78/192, lr:0.000020, giou_loss:  33.61, conf_loss: 945.67, prob_loss:  63.40, total_loss:1042.68
epoch: 0 step:   79/192, lr:0.000021, giou_loss:  39.32, conf_loss: 930.53, prob_loss:  73.47, total_loss:1043.32
epoch: 0 step:   80/192, lr:0.000021, giou_loss:  22.84, conf_loss: 899.33, prob_loss:  44.18, total_loss: 966.36
epoch: 0 step:   81/192, lr:0.000021, giou_loss:  22.52, conf_loss: 890.44, prob_loss:  44.19, total_loss: 957.16
epoch: 0 step:   82/192, lr:0.000021, giou_loss:  40.88, conf_loss: 916.80, prob_loss:  

epoch: 0 step:  146/192, lr:0.000038, giou_loss:  30.63, conf_loss: 606.59, prob_loss:  41.97, total_loss: 679.19
epoch: 0 step:  147/192, lr:0.000038, giou_loss:  40.04, conf_loss: 603.32, prob_loss:  53.37, total_loss: 696.74
epoch: 0 step:  148/192, lr:0.000039, giou_loss:  37.66, conf_loss: 601.98, prob_loss:  51.97, total_loss: 691.60
epoch: 0 step:  149/192, lr:0.000039, giou_loss:  18.92, conf_loss: 594.42, prob_loss:  28.08, total_loss: 641.41
epoch: 0 step:  150/192, lr:0.000039, giou_loss:  24.41, conf_loss: 595.04, prob_loss:  32.09, total_loss: 651.55
epoch: 0 step:  151/192, lr:0.000039, giou_loss:  21.23, conf_loss: 598.92, prob_loss:  28.31, total_loss: 648.46
epoch: 0 step:  152/192, lr:0.000040, giou_loss:  26.06, conf_loss: 589.28, prob_loss:  33.15, total_loss: 648.49
epoch: 0 step:  153/192, lr:0.000040, giou_loss:  33.78, conf_loss: 595.35, prob_loss:  44.80, total_loss: 673.94
epoch: 0 step:  154/192, lr:0.000040, giou_loss:  29.98, conf_loss: 584.14, prob_loss:  

epoch: 1 step:   26/192, lr:0.000057, giou_loss:  39.06, conf_loss: 386.78, prob_loss:  25.16, total_loss: 451.00
epoch: 1 step:   27/192, lr:0.000057, giou_loss:  25.10, conf_loss: 392.40, prob_loss:  22.94, total_loss: 440.45
epoch: 1 step:   28/192, lr:0.000057, giou_loss:  23.78, conf_loss: 375.61, prob_loss:  13.00, total_loss: 412.39
epoch: 1 step:   29/192, lr:0.000058, giou_loss:  25.59, conf_loss: 375.74, prob_loss:  21.01, total_loss: 422.34
epoch: 1 step:   30/192, lr:0.000058, giou_loss:  31.04, conf_loss: 374.31, prob_loss:  22.95, total_loss: 428.31
epoch: 1 step:   31/192, lr:0.000058, giou_loss:  28.01, conf_loss: 372.01, prob_loss:  19.85, total_loss: 419.88
epoch: 1 step:   32/192, lr:0.000058, giou_loss:  22.36, conf_loss: 369.74, prob_loss:  14.88, total_loss: 406.97
epoch: 1 step:   33/192, lr:0.000059, giou_loss:  40.09, conf_loss: 377.33, prob_loss:  28.43, total_loss: 445.85
epoch: 1 step:   34/192, lr:0.000059, giou_loss:  29.01, conf_loss: 361.98, prob_loss:  

epoch: 1 step:   98/192, lr:0.000076, giou_loss:  22.97, conf_loss: 226.76, prob_loss:  13.00, total_loss: 262.73
epoch: 1 step:   99/192, lr:0.000076, giou_loss:  20.08, conf_loss: 228.53, prob_loss:  11.26, total_loss: 259.87
epoch: 1 step:  100/192, lr:0.000076, giou_loss:  26.01, conf_loss: 226.53, prob_loss:  13.23, total_loss: 265.78
epoch: 1 step:  101/192, lr:0.000076, giou_loss:  22.58, conf_loss: 217.25, prob_loss:   7.19, total_loss: 247.02
epoch: 1 step:  102/192, lr:0.000077, giou_loss:  21.08, conf_loss: 217.70, prob_loss:  13.64, total_loss: 252.42
epoch: 1 step:  103/192, lr:0.000077, giou_loss:  29.83, conf_loss: 224.61, prob_loss:  15.37, total_loss: 269.81
epoch: 1 step:  104/192, lr:0.000077, giou_loss:  26.68, conf_loss: 229.82, prob_loss:  15.02, total_loss: 271.52
epoch: 1 step:  105/192, lr:0.000077, giou_loss:  25.71, conf_loss: 227.00, prob_loss:  13.66, total_loss: 266.36
epoch: 1 step:  106/192, lr:0.000078, giou_loss:  26.30, conf_loss: 215.14, prob_loss:  

...

epoch:21 step:  146/192, lr:0.000090, giou_loss:   4.92, conf_loss:   1.89, prob_loss:   2.02, total_loss:   8.83
epoch:21 step:  147/192, lr:0.000090, giou_loss:   8.71, conf_loss:   0.88, prob_loss:   3.78, total_loss:  13.37
epoch:21 step:  148/192, lr:0.000090, giou_loss:   9.12, conf_loss:   3.46, prob_loss:   3.03, total_loss:  15.61
epoch:21 step:  149/192, lr:0.000090, giou_loss:  11.35, conf_loss:   1.78, prob_loss:   3.81, total_loss:  16.94
epoch:21 step:  150/192, lr:0.000090, giou_loss:   9.45, conf_loss:   1.75, prob_loss:   3.38, total_loss:  14.58
epoch:21 step:  151/192, lr:0.000090, giou_loss:  10.14, conf_loss:   1.45, prob_loss:   3.83, total_loss:  15.42
epoch:21 step:  152/192, lr:0.000090, giou_loss:  12.80, conf_loss:   2.08, prob_loss:   4.53, total_loss:  19.41
epoch:21 step:  153/192, lr:0.000090, giou_loss:   9.06, conf_loss:   3.61, prob_loss:   3.16, total_loss:  15.84
epoch:21 step:  154/192, lr:0.000090, giou_loss:   9.63, conf_loss:   0.88, prob_loss:  

epoch:22 step:   26/192, lr:0.000090, giou_loss:  10.79, conf_loss:   0.98, prob_loss:   4.22, total_loss:  15.99
epoch:22 step:   27/192, lr:0.000090, giou_loss:  11.36, conf_loss:   0.90, prob_loss:   4.21, total_loss:  16.48
epoch:22 step:   28/192, lr:0.000090, giou_loss:   7.18, conf_loss:   0.82, prob_loss:   2.68, total_loss:  10.68
epoch:22 step:   29/192, lr:0.000090, giou_loss:   9.64, conf_loss:   0.76, prob_loss:   4.06, total_loss:  14.46
epoch:22 step:   30/192, lr:0.000090, giou_loss:   8.91, conf_loss:   1.34, prob_loss:   3.27, total_loss:  13.53
epoch:22 step:   31/192, lr:0.000090, giou_loss:   9.59, conf_loss:   1.07, prob_loss:   3.85, total_loss:  14.51
epoch:22 step:   32/192, lr:0.000090, giou_loss:  17.91, conf_loss:   1.68, prob_loss:   6.34, total_loss:  25.93
epoch:22 step:   33/192, lr:0.000090, giou_loss:  11.80, conf_loss:   1.40, prob_loss:   4.83, total_loss:  18.04
epoch:22 step:   34/192, lr:0.000090, giou_loss:  11.99, conf_loss:   0.69, prob_loss:  

epoch:22 step:   98/192, lr:0.000090, giou_loss:   9.47, conf_loss:   0.80, prob_loss:   3.37, total_loss:  13.63
epoch:22 step:   99/192, lr:0.000090, giou_loss:  11.94, conf_loss:   1.21, prob_loss:   4.24, total_loss:  17.39
epoch:22 step:  100/192, lr:0.000090, giou_loss:   9.76, conf_loss:   0.79, prob_loss:   3.55, total_loss:  14.10
epoch:22 step:  101/192, lr:0.000090, giou_loss:   6.63, conf_loss:   0.63, prob_loss:   3.25, total_loss:  10.50
epoch:22 step:  102/192, lr:0.000090, giou_loss:   6.69, conf_loss:   2.17, prob_loss:   2.80, total_loss:  11.66
epoch:22 step:  103/192, lr:0.000090, giou_loss:   8.28, conf_loss:   1.07, prob_loss:   3.61, total_loss:  12.95
epoch:22 step:  104/192, lr:0.000090, giou_loss:  11.95, conf_loss:   1.04, prob_loss:   4.31, total_loss:  17.31
epoch:22 step:  105/192, lr:0.000090, giou_loss:   9.50, conf_loss:   1.10, prob_loss:   3.59, total_loss:  14.19
epoch:22 step:  106/192, lr:0.000090, giou_loss:   8.67, conf_loss:   1.34, prob_loss:  

epoch:22 step:  170/192, lr:0.000089, giou_loss:   7.69, conf_loss:   1.30, prob_loss:   3.24, total_loss:  12.23
epoch:22 step:  171/192, lr:0.000089, giou_loss:   9.44, conf_loss:   1.96, prob_loss:   4.07, total_loss:  15.47
epoch:22 step:  172/192, lr:0.000089, giou_loss:   8.03, conf_loss:   0.94, prob_loss:   2.86, total_loss:  11.83
epoch:22 step:  173/192, lr:0.000089, giou_loss:   9.76, conf_loss:   1.10, prob_loss:   3.47, total_loss:  14.33
epoch:22 step:  174/192, lr:0.000089, giou_loss:   8.10, conf_loss:   2.79, prob_loss:   3.01, total_loss:  13.89
epoch:22 step:  175/192, lr:0.000089, giou_loss:  14.88, conf_loss:   1.10, prob_loss:   4.90, total_loss:  20.87
epoch:22 step:  176/192, lr:0.000089, giou_loss:  13.72, conf_loss:   1.37, prob_loss:   4.37, total_loss:  19.46
epoch:22 step:  177/192, lr:0.000089, giou_loss:   6.83, conf_loss:   3.26, prob_loss:   2.35, total_loss:  12.44
epoch:22 step:  178/192, lr:0.000089, giou_loss:  11.62, conf_loss:   0.98, prob_loss:  

KeyboardInterrupt: 

In [3]:
yolo.save_weights(save_directory)